In [113]:
import numpy as np
from scipy.linalg import norm, eig
from scipy.sparse import dok_matrix
import pandas as pd

In [44]:
def get_A(filename, N):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            try:
                data.append(tuple((int(x) for x in line.strip().split())))
            except:
                pass
    A = dok_matrix((N, N))
    for edge in data:
        A[edge[0], edge[1]] = 1
    return A
        

A = get_adj('../matrix.txt', 8).toarray()

In [45]:
def get_K(A):
    D_diag = np.sum(A, 1)
    for i, rowsum in enumerate(D_diag):
        if rowsum == 0:
            A[i,:] += 1
            D_diag[i] = sum(A[i,:])
    return (np.diag(1 / D_diag)@A).T

In [54]:
K = get_K(A)

In [79]:
def get_SS(A, N=None, d=0.85, tol=1e-5):
    if N is None:
        (N, N) = np.shape(A)
        K = get_K(A)
    else:
        K = get_K(A[:N,:N])
    p = np.random.random(N)
    while True:
        p_prime = d * K@p + ((1 - d) / N) * np.ones(N)
        if la.norm(p_prime - p) < tol:
            break
        p = p_prime
    return p_prime

get_SS(A)

array([ 0.43870407,  0.02171029,  0.02786154,  0.02171029,  0.02171029,
        0.02786154,  0.04585394,  0.39461572])

In [78]:
def get_SS_eig_method(A, N=None, d=0.85):
    if N is None:
        (N, N) = np.shape(A)
        K = get_K(A)
    else:
        K = get_K(A[:N,:N])
    B = d * K + ((1 - d) / N) * np.ones(N)
    eigs, vecs = eig(B)
    return eigs[0], vecs[:,0] / sum(vecs[:,0])

get_SS_eig_method(A)

((0.999999999999999+0j),
 array([ 0.43869288,  0.02171029,  0.02786154,  0.02171029,  0.02171029,
         0.02786154,  0.04585394,  0.39459924]))

In [140]:
def comp_ranking(filename):
    data = pd.read_csv(filename, header=0)
    teams = pd.Series(data.values.ravel()).unique()
    A = pd.DataFrame(np.zeros((teams.size, teams.size)), index=teams, columns=teams)
    for row in data.values:
        A.loc[row[1], row[0]] = 1
    return sorted(zip(teams, get_SS(A, d=0.7)), key=lambda x: -x[1])[:5]

comp_ranking('../ncaa2013.csv')

[('Duke', 0.0096615547966975406),
 ('Butler', 0.0085389943342128131),
 ('Louisville', 0.0084984316239761094),
 ('Illinois', 0.0083358476525664162),
 ('Indiana', 0.0082256987111321758)]